This notebook trains a reward model with DeepSpeed Chat using OPT-350 as a base model.
This is the second step for training an instruct LLMs with DeepSpeed Chat.
The details of the steps are explained in this article: [Train Instruct LLMs On Your GPU with DeepSpeed Chat — Step #2: Training a Reward Model](https://kaitchup.substack.com/p/train-instruct-llms-on-your-gpu-with-1e1)

You can find more details on supervised fine-tuning using DeepSpeed Chat in this article: [Train Instruct LLMs On Your GPU with DeepSpeed Chat — Step #1: Supervised Fine-tuning](https://kaitchup.substack.com/p/train-instruct-llms-on-your-gpu-with)

In [ ]:
!pip install deepspeed>=0.9.0

!git clone https://github.com/microsoft/DeepSpeedExamples.git
%cd DeepSpeedExamples/applications/DeepSpeed-Chat/
!pip install -r requirements.txt

Cloning into 'DeepSpeedExamples'...
remote: Enumerating objects: 8652, done.
remote: Counting objects: 100% (1761/1761), done.
remote: Compressing objects: 100% (307/307), done.
remote: Total 8652 (delta 1562), reused 1503 (delta 1422), pack-reused 6891
Receiving objects: 100% (8652/8652), 22.37 MiB | 21.90 MiB/s, done.
Resolving deltas: 100% (4891/4891), done.
/content/DeepSpeedExamples/applications/DeepSpeed-Chat
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

Train a reward model using opt-350 as a base model. It requires at least 41 GB of CPU RAM and 10 GB of VRAM.
The reward model is saved in a directory named rm_ds.

In [ ]:
%cd training/step2_reward_model_finetuning/
!deepspeed --num_gpus 1 main.py --model_name_or_path facebook/opt-350m \
   --data_path Dahoas/rm-static Dahoas/full-hh-rlhf Dahoas/synthetic-instruct-gptj-pairwise yitingxie/rlhf-reward-datasets \
   --data_split 2,4,4 \
   --gradient_checkpointing \
   --num_train_epochs 1 \
   --per_device_eval_batch_size 4 \
   --per_device_train_batch_size 4 \
   --seed 1234 \
   --num_padding_at_beginning 1 --weight_decay 0.1 --disable_dropout --gradient_accumulation_steps 16 --zero_stage 0 \
   --deepspeed --output_dir ./rm_ds/

/content/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step2_reward_model_finetuning/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step2_reward_model_finetuning
[2023-09-14 11:17:02,998] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-09-14 11:17:08.240743: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-09-14 11:17:09,231] [WARNING] [runner.py:203:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-09-14 11:17:09,231] [INFO] [runner.py:570:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None main.py --model_name_or_path facebook/opt-350m --data_path Dahoas/rm-static Dahoas/full-hh-rlhf Dahoas/synthetic-instruct-gptj-pairwise yitingxie/rlhf-reward-datasets --data_split 2,4,4 --gradient_checkpointing -

The following cell runs the evaluation of the model.
Note: If you have an error, follow the instructions at the end of this article: [Train Instruct LLMs On Your GPU with DeepSpeed Chat — Step #2: Training a Reward Model](https://kaitchup.substack.com/p/train-instruct-llms-on-your-gpu-with-1e1)

In [ ]:
%cd training/step2_reward_model_finetuning/
!python rw_eval.py  --model_name_or_path kaitchup/OPT-350M-RM-DSChat

[Errno 2] No such file or directory: 'training/step2_reward_model_finetuning/'
/content/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step2_reward_model_finetuning
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
[2023-09-14 15:41:03,253] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-09-14 15:41:08.180435: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50272. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to 